In [1]:
# Cell 1 — Imports & connect
import sqlite3
import pandas as pd

# open (or create) your SQLite database
conn = sqlite3.connect('economic_data.db')


In [2]:
# Cell 2 — Read each CSV and push to SQL as raw tables
datasets = [
    ('gdp',        '../data/gdp.csv'),
    ('emp',        '../data/emp.csv'),
    ('gdp_growth', '../data/gdp_growth.csv'),
]

for level, path in datasets:
    print(f"→ Loading raw table: {level}_raw from {path!r}")
    df = pd.read_csv(path, skiprows=4)  # adjust skiprows only if necessary
    df.to_sql(f"{level}_raw", conn, if_exists='replace', index=False)

print("All raw tables loaded into SQLite.")


→ Loading raw table: gdp_raw from '../data/gdp.csv'
→ Loading raw table: emp_raw from '../data/emp.csv'
→ Loading raw table: gdp_growth_raw from '../data/gdp_growth.csv'
All raw tables loaded into SQLite.


In [3]:
import sqlite3

# 1) Connect & get a cursor
conn = sqlite3.connect('economic_data.db')
cur  = conn.cursor()

# 2) Define your filters (you may generate these programmatically, but we won't loop over tables)
target_countries = ['China','Japan','South Korea','United States','United Kingdom','Canada']
countries_sql = ", ".join(f"'{c}'" for c in target_countries)

years    = [str(y) for y in range(1990, 2024)]
columns  = ['Country Name', 'Indicator Name'] + years
col_clause = ", ".join(f'"{c}"' for c in columns)

# 3) Filter gdp_raw → gdp_filtered, drop old, rename
cur.execute(f"""
    CREATE TABLE gdp_filtered AS
    SELECT {col_clause}
      FROM gdp_raw
     WHERE "Country Name" IN ({countries_sql});
""")
cur.execute("DROP TABLE gdp_raw;")
cur.execute("ALTER TABLE gdp_filtered RENAME TO gdp_raw;")
conn.commit()

# 4) Filter emp_raw → emp_filtered, drop old, rename
cur.execute(f"""
    CREATE TABLE emp_filtered AS
    SELECT {col_clause}
      FROM emp_raw
     WHERE "Country Name" IN ({countries_sql});
""")
cur.execute("DROP TABLE emp_raw;")
cur.execute("ALTER TABLE emp_filtered RENAME TO emp_raw;")
conn.commit()

# 5) Filter gdp_growth_raw → gdp_growth_filtered, drop old, rename
cur.execute(f"""
    CREATE TABLE gdp_growth_filtered AS
    SELECT {col_clause}
      FROM gdp_growth_raw
     WHERE "Country Name" IN ({countries_sql});
""")
cur.execute("DROP TABLE gdp_growth_raw;")
cur.execute("ALTER TABLE gdp_growth_filtered RENAME TO gdp_growth_raw;")
conn.commit()

# 6) Verify first 5 rows of each
for tbl in ['gdp_raw','emp_raw','gdp_growth_raw']:
    print(f"\n--- {tbl} ---")
    cur.execute(f"SELECT {col_clause} FROM {tbl} LIMIT 5;")
    rows = cur.fetchall()
    # print column headers then rows
    print(columns)
    for row in rows:
        print(row)

# 7) Close connection
conn.close()



--- gdp_raw ---
['Country Name', 'Indicator Name', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
('Canada', 'GDP per capita (constant 2015 US$)', 31700.54627, 30654.33314, 30563.37882, 31032.7546, 32074.15624, 32595.45152, 32801.30185, 33867.00791, 34894.61343, 36391.53533, 37906.86012, 38200.45611, 38921.6673, 39270.02345, 40108.75874, 41006.22293, 41663.51231, 42106.87243, 42067.5687, 40376.41537, 41164.33991, 42043.64196, 42320.64313, 42851.17053, 43643.23565, 43594.1941, 43551.3426, 44339.38867, 44907.34368, 45100.29149, 42366.12852, 44359.61673, 45227.14474, 44468.7475)
('China', 'GDP per capita (constant 2015 US$)', 905.0324572, 975.4629156, 1100.646116, 1239.129373, 1384.930157, 1520.029469, 1653.433789, 1787.766977, 1909.622328, 2038.206474, 2193.896866, 

We assemble three indicators from the World Bank’s World Development Indicators:

GDP per capita (constant 2015 US$)

Employment to population ratio

Annual GDP growth (%)

For each, we focus on six high- and middle-income economies (China, Japan, South Korea, the United States, United Kingdom, Canada) over the period 1990–2023. We will merge these into a single country–year panel, clean and format, and then explore summary statistics for each indicator.

In [5]:
# Cell 3 — Reconnect to the database
import sqlite3
import pandas as pd

conn = sqlite3.connect('economic_data.db')

# Read all three tables back into pandas
print("Reading tables into pandas DataFrames...")
gdp_df        = pd.read_sql_query("SELECT * FROM gdp_raw", conn)
emp_df        = pd.read_sql_query("SELECT * FROM emp_raw", conn)
gdp_growth_df = pd.read_sql_query("SELECT * FROM gdp_growth_raw", conn)

# Cell 4 — Data Cleaning
print("\nCleaning data...")

# Melt the dataframes (wide to long format) for easier merging later
def melt_df(df, value_name):
    id_vars = ['Country Name', 'Indicator Name']
    value_vars = [col for col in df.columns if col not in id_vars]
    return df.melt(id_vars=id_vars, value_vars=value_vars, 
                   var_name='Year', value_name=value_name)

gdp_long        = melt_df(gdp_df, 'GDP')
emp_long        = melt_df(emp_df, 'Employment')
gdp_growth_long = melt_df(gdp_growth_df, 'GDP_Growth')

# Make sure 'Year' is an integer and value columns are numeric
def clean_numeric(df, value_col):
    df['Year'] = df['Year'].astype(int)
    df[value_col] = pd.to_numeric(df[value_col], errors='coerce')

clean_numeric(gdp_long, 'GDP')
clean_numeric(emp_long, 'Employment')
clean_numeric(gdp_growth_long, 'GDP_Growth')

# Cell 5 — Merge Tables Together
print("\nMerging datasets...")

merged_df = gdp_long.merge(emp_long,        on=['Country Name','Indicator Name','Year'], how='outer')
merged_df = merged_df.merge(gdp_growth_long, on=['Country Name','Indicator Name','Year'], how='outer')

# Because 'Indicator Name' can be different across tables, you can drop it after confirming
merged_df.drop(columns=['Indicator Name'], inplace=True)

# Optional: Reorder columns
merged_df = merged_df[['Country Name', 'Year', 'GDP', 'Employment', 'GDP_Growth']]

# Cell 6 — Save Cleaned DataFrame into SQL
print("\nSaving merged cleaned data into database as 'econ_cleaned'...")

merged_df.to_sql('econ_cleaned', conn, if_exists='replace', index=False)

# Cell 7 — Descriptive Statistics Summary
print("\nGenerating summary statistics...")

summary_stats = merged_df.groupby('Country Name').agg({
    'GDP': ['count', 'mean', 'std', 'min', 'max'],
    'Employment': ['count', 'mean', 'std', 'min', 'max'],
    'GDP_Growth': ['count', 'mean', 'std', 'min', 'max'],
})

# Clean up column names
summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]

# Display
print(summary_stats)

# Optional: Save summary statistics to a CSV
summary_stats.to_csv('summary_statistics.csv')

# Final Step — Close connection
conn.close()

print("\nAll tasks completed!")


Reading tables into pandas DataFrames...

Cleaning data...

Merging datasets...

Saving merged cleaned data into database as 'econ_cleaned'...

Generating summary statistics...
                GDP_count      GDP_mean      GDP_std       GDP_min  \
Country Name                                                         
Canada                 34  39357.078405  4856.836357  30563.378820   
China                  34   5073.229508  3583.630683    905.032457   
Japan                  34  32770.738504  2371.821778  28422.213120   
United Kingdom         34  40583.327987  5456.617589  30441.481350   
United States          34  51521.551093  7640.364126  38637.839810   

                    GDP_max  Employment_count  Employment_mean  \
Country Name                                                     
Canada          45227.14474                33        60.712303   
China           12175.19611                33        69.123242   
Japan           36990.33011                33        59.238939   
Un